In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd 
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import random
import os
print(os.listdir("../input/dogs-vs-cats"))

In [ ]:
IMAGE_WIDTH=64
IMAGE_HEIGHT=64
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
TRAIN_DIRECTORY="/kaggle/working/train/"
TEST_DIRECTORY="/kaggle/working/test1"

In [ ]:
def extract_files(source_path, target_path):
    zip_ref = zipfile.ZipFile(source_path,'r')
    zip_ref.extractall(target_path)
    zip_ref.close()

In [ ]:
def extraction():
    extract_files('/kaggle/input/dogs-vs-cats/test1.zip','/kaggle/working/')
    extract_files('/kaggle/input/dogs-vs-cats/train.zip','/kaggle/working/')

In [ ]:
extraction()

def get_filenames(directory):
    filenames=(os.listdir(directory))
    return filenames


def load_data(filenames,directory):
#     i=500   #for testing purpose
    i=len(filenames);
    X=[]
    y=[]
    for name in filenames:
        img=mpimg.imread(os.path.join(directory,name))
        X.append(cv2.resize(img,IMAGE_SIZE))
        cat=name.split('.')[0]
        if(cat=='dog'):
            y.append(0)
        else:
            y.append(1)
        i-=1
        if(i<=0):
            break
    return X,y

In [ ]:
filenames=get_filenames(TRAIN_DIRECTORY)
X,Y=load_data(filenames,TRAIN_DIRECTORY)

In [ ]:
temp_x=np.array(X)
temp_y=np.array(Y).reshape(len(Y),1);
x_train,x_test,y_train,y_test=train_test_split(temp_x,temp_y);
x_train=x_train.reshape(x_train.shape[0],-1);
x_test=x_test.reshape(x_test.shape[0],-1);
x_train=x_train.T;
x_test=x_test.T;
x_train=x_train/255;
x_test=x_test/255;
y_train=y_train.T;
y_test=y_test.T;
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
def random_mini_batches(X,Y,mini_batch_size=64):
    m=X.shape[1];
    mini_batches=[];
    permutation=list(np.random.permutation(m))
    X=X[:,permutation];
    Y=Y[:,permutation];
    for i in range(m//mini_batch_size):
        mini_batch_x=X[:, i*mini_batch_size:(i+1)*mini_batch_size ];
        mini_batch_y=Y[:, i*mini_batch_size:(i+1)*mini_batch_size ];
        mini_batches.append((mini_batch_x,mini_batch_y));
        
    if m%mini_batch_size!=0:
        mini_batch_x=X[:, (m//mini_batch_size)*mini_batch_size: ];
        mini_batch_y=Y[:, (m//mini_batch_size)*mini_batch_size: ];
        mini_batches.append((mini_batch_x,mini_batch_y));
    
    return mini_batches;

In [ ]:
layer_dims=[x_train.shape[0],20,20,7,5,1];
def initialize_parameters():
    parameters={};
    np.random.seed(1);
    for i in range(1,len(layer_dims)):
        parameters['W'+str(i)]=np.random.randn(layer_dims[i],layer_dims[i-1])/np.sqrt(layer_dims[i-1]);
        parameters['b'+str(i)]=np.zeros((layer_dims[i],1));
    return parameters;

In [ ]:
def linear_forward(A_prev,W,b):
#     print('W',W);
#     print('b',b);
#     print('Input_to_layer',A_prev);
    Z=np.dot(W,A_prev)+b;
    linear_cache=(W,b,A_prev);
    return (Z,linear_cache);
    

In [ ]:
def linear_activation_forward(A_prev,W,b,activation):
    Z,linear_cache=linear_forward(A_prev,W,b);
#     print("Z",Z);
    A=None;
    if(activation=="sigmoid"):
        A=1/(1+np.exp(-Z));
    else:
        A=np.maximum(0,Z);
    cache=(linear_cache,Z);
    return A,cache;
        

In [ ]:
def L_model_forward(X,parameters):
    caches=[];
    A_prev=X;
    for l in range(1,len(layer_dims)-1):
        A,temp_cache=linear_activation_forward(A_prev,parameters["W"+str(l)],parameters["b"+str(l)], "relu");
        A_prev=A;
        caches.append(temp_cache);
    A,temp_cache=linear_activation_forward(A_prev,parameters["W"+str(len(layer_dims)-1)],parameters["b"+str(len(layer_dims)-1)], "sigmoid");
    caches.append(temp_cache);
    return A,caches;
        

In [ ]:
def compute_cost(AL,Y):
    m = Y.shape[1]
    cost = -np.sum((Y*np.log(AL) + (1-Y)*np.log(1-AL)),keepdims=True)/m;
    cost = np.squeeze(cost)
    return cost

In [ ]:
def linear_backward(dz,cache):
    W,b,A_prev=cache;
    m = A_prev.shape[1]
    
#     print("W.T",W.T);
#     print("dz",dz);
#     print("A_prev.T",A_prev.T);
    dA_prev=np.dot(W.T,dz);
    dw=np.dot(dz,A_prev.T)/m;
    db=np.sum(dz,axis=1,keepdims=True)/m;
    return dA_prev,dw,db;

In [ ]:
def linear_activation_backward(dA,cache,activation):
    linear_cache,activation_cache=cache;
    dz=None;
    if(activation=="sigmoid"):
#         print("Activation_cache",activation_cache);
        A=1/(1+np.exp(-activation_cache));
#         print("dA",dA);
#         print("A",A);
        dz=dA*A*(1-A);
    else:
        z=activation_cache;
#         print("Activation_cache",activation_cache);
        dz=dA.copy();
        dz[z<=0]=0;
    return linear_backward(dz,linear_cache);

In [ ]:
def L_model_backward(AL,Y,Caches):
    L=len(layer_dims)-1; # no of layers=3;
    grads={};
#     print("Y",Y);
#     print("AL",AL);
    dAL=-Y/AL + (1-Y)/(1-AL);
    current_cache = Caches[L-1];
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL,current_cache,"sigmoid");
    for l in reversed(range(L-1)):
        current_cache = Caches[l];
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+1)],current_cache,"relu");
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads;

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    L=len(layer_dims)-1; # no of layers=3;
    for l in range(L):
        parameters["W" + str(l+1)] =  parameters["W" + str(l+1)]- learning_rate*grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*grads["db" + str(l+1)]
    return parameters

In [ ]:
def L_Layer_model(X,Y,learning_rate=0.07,epochs=1000,print_cost=False):
    costs=[];
    parameters=initialize_parameters();
    mini_batches=random_mini_batches(X,Y,64);
    for i in range(0,epochs+1):
        cost_total=0;
        for mini_batch in mini_batches:
            mini_batch_x,mini_batch_y=mini_batch;
            AL, caches = L_model_forward(mini_batch_x,parameters);
            cost_total =cost_total+ compute_cost(AL, mini_batch_y)
            grads = L_model_backward(AL, mini_batch_y, caches);
            parameters = update_parameters(parameters, grads, learning_rate)
        cost_total=cost_total/X.shape[1];
        if(i%100==0):
            print("Cost at",i,"epoch",cost_total);
        costs.append(cost_total);
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    return parameters;
    

In [ ]:
parameters = L_Layer_model(x_train, y_train,epochs = 500, print_cost = True)

In [ ]:
def predict(X, y, parameters):
    """
    This function is used to predict the results of a  L-layer neural network.
    
    Arguments:
    X -- data set of examples you would like to label
    parameters -- parameters of the trained model
    
    Returns:
    p -- predictions for the given dataset X
    """
    
    m = X.shape[1]
    n = len(parameters) // 2 # number of layers in the neural network
    p = np.zeros((1,m))
    
    # Forward propagation
    probas, caches = L_model_forward(X, parameters)

    
    # convert probas to 0/1 predictions
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
    
    #print results
    #print ("predictions: " + str(p))
    #print ("true labels: " + str(y))
    print("Accuracy: "  + str(np.sum((p == y)/m)))
        
    return p

In [ ]:
pred_train = predict(x_train, y_train, parameters)

In [ ]:
pred_train = predict(x_test, y_test, parameters)